# Traducción de español a inglés con un Transformer secuencia a secuencia

**Autores:** Alina Rojas y Mar Iborra<br>
**Adaptado de:** [Ejemplo de Keras - Neural Machine Translation with Transformer](https://keras.io/examples/nlp/neural_machine_translation_with_transformer/)<br>
**Basado en el trabajo original de:** [fchollet](https://twitter.com/fchollet)<br>
**Fecha de creación de la adaptación:** 2023/12/31<br>
**Última modificación:** 2024/01/12<br>
**Descripción:** Este proyecto consiste en la implementación de un Transformer secuencia a secuencia para realizar tareas de traducción automática de español a inglés. Se ha adaptado del ejemplo de Keras, que fue diseñado originalmente para la traducción de inglés a español. Las adaptaciones incluyen ajustes en la preparación de los datos, la vectorización y la estructura del modelo para adecuarse a la traducción de español a inglés, basándose en las técnicas y el código proporcionados por F. Chollet.

## Introducción

En este proyecto, desarrollaremos un modelo Transformer secuencia a secuencia, el cual será entrenado para tareas de traducción automática de español a inglés.

Este proyecto implica:

- Vectorización de texto utilizando la capa `TextVectorization` de Keras.
- Implementación de una capa `TransformerEncoder`, una capa `TransformerDecoder` y una capa `PositionalEmbedding`.
- Preparación de datos para entrenar un modelo secuencia a secuencia.
- Uso del modelo entrenado para generar traducciones de oraciones de entrada nunca antes vistas (inferencia secuencia a secuencia).

El código utilizado se baso y adapto del libro [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition) (capítulo 11: Deep learning for text).



### ¿Qué es un modelo Transformer secuencia a secuencia?

Un modelo Transformer secuencia a secuencia es una arquitectura avanzada de aprendizaje profundo utilizada principalmente para tareas de traducción automática. Consiste en dos componentes clave: el codificador (encoder) y el decodificador (decoder). El codificador procesa la oración de entrada, mientras que el decodificador genera la traducción correspondiente. Este modelo es altamente eficaz para manejar secuencias de texto, gracias a su habilidad para captar dependencias a larga distancia y su capacidad para el procesamiento paralelo de los datos.

## Configuración

In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import pathlib
import random
import string
import re
import numpy as np

import tensorflow.data as tf_data
import tensorflow.strings as tf_strings

import keras
from keras import layers
from keras.layers import TextVectorization

import tensorflow as tf

#### Cambios respecto al código original:

En el código original, se hacía uso de `from keras import ops`, pero en nuestra adaptación, eliminamos esta importación. El cambio se debió a que `ops`, un submódulo de operaciones de bajo nivel, no es parte del módulo `keras` estándar, sino de TensorFlow (`tensorflow`). Por lo tanto, en lugar de utilizar `keras.ops`, optamos por usar directamente TensorFlow (`import tensorflow as tf`), que proporciona todas las funciones y operaciones necesarias.

Este ajuste garantiza que todas las operaciones de TensorFlow se manejen de manera eficiente y compatible, evitando problemas de importación y compatibilidad que podrían surgir al intentar acceder a operaciones específicas de TensorFlow a través de Keras.

## Descargando los Datos

Trabajaremos con un conjunto de datos de traducción de inglés a español proporcionado por [Anki](https://www.manythings.org/anki/). Aunque el conjunto de datos original está diseñado para la traducción de inglés a español, adaptaremos el uso de estos datos para realizar traducciones en la dirección opuesta, de español a inglés. Descarguemos el conjunto de datos:

In [2]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

2638744/2638744 [==============================] - 0s 0us/step


Este paso garantiza que tenemos acceso a un conjunto de datos adecuado que, aunque originalmente está enfocado en la traducción de inglés a español, será adaptado en nuestro código para soportar traducciones de español a inglés.

## Procesamiento de los Datos

Cada línea del conjunto de datos contiene una oración en inglés y su correspondiente traducción en español. En el enfoque original, la oración en inglés es la *secuencia de origen* y la española es la *secuencia objetivo*. Sin embargo, para nuestro proyecto, invertiremos este enfoque: la oración en español será nuestra secuencia de origen y la oración en inglés será la secuencia objetivo. Prependemos el token `"[start]"` y añadimos el token `"[end]"` a la oración en inglés.

In [3]:
with open(text_file, encoding='utf-8') as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    eng = "[start] " + eng + " [end]"
    text_pairs.append((spa, eng))

Este cambio en la preparación de los datos es crucial para adaptar el conjunto de datos a nuestro objetivo de traducción de español a inglés. Al modificar las secuencias objetivo, aseguramos que el modelo aprenda a traducir del español al inglés, lo cual es la finalidad principal de nuestro proyecto.

Algunos ejemplos de cómo lucen nuestros pares de oraciones:

In [4]:
for _ in range(5):
    print(random.choice(text_pairs))

('Los demás no le hicieron caso a su advertencia.', '[start] The others paid no attention to her warning. [end]')
('Tom camina despacio.', '[start] Tom walks slowly. [end]')
('Ella es muy amable con nosotras.', '[start] She is very kind to us. [end]')
('A veces no le entiendo.', "[start] I don't understand him sometimes. [end]")
('Tom compró un par de zapatos baratos, pero no le duraron mucho tiempo.', "[start] Tom bought a pair of cheap shoes, but they didn't last very long. [end]")


En cada par, la primera oración está en español y actúa como nuestra secuencia de entrada (origen), y la segunda oración está en inglés con los tokens `"[start]"` y `"[end]"` añadidos, sirviendo como nuestra secuencia objetivo. Estos ejemplos muestran cómo estamos preparando los datos para entrenar nuestro modelo de traducción de español a inglés.

Ahora, dividiremos los pares de oraciones en un conjunto de entrenamiento, un conjunto de validación y un conjunto de prueba.

In [5]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


Esta división nos permite entrenar nuestro modelo en una variedad de oraciones, validar su rendimiento en un conjunto separado y, finalmente, evaluar su capacidad de generalización con oraciones que no ha visto antes en el conjunto de prueba. Este enfoque es esencial para desarrollar un modelo robusto y eficaz de traducción de español a inglés.

## Vectorización de los Datos Textuales

Utilizaremos dos instancias de la capa `TextVectorization` para vectorizar los datos textuales (una para el español y otra para el inglés), es decir, para convertir las cadenas originales en secuencias de enteros donde cada entero representa el índice de una palabra en un vocabulario.

La capa para el inglés usará la estandarización y el esquema de división predeterminados de `TextVectorization` (eliminar caracteres de puntuación y dividir en espacios en blanco), mientras que la capa para el español usará una estandarización personalizada, en la que añadiremos el carácter `"¿"` al conjunto de caracteres de puntuación que se eliminarán.

Nota del trabajo original: en un modelo de traducción automática de grado productivo, no recomendaría eliminar los caracteres de puntuación en ninguno de los idiomas. En su lugar, recomendaría convertir cada carácter de puntuación en su propio token, lo que se podría lograr proporcionando una función `split` personalizada a la capa `TextVectorization`.

In [6]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64

def custom_standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    return tf_strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")

spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
    standardize=custom_standardization,
)
eng_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
)
train_spa_texts = [pair[0] for pair in train_pairs]
train_eng_texts = [pair[1] for pair in train_pairs]
spa_vectorization.adapt(train_spa_texts)
eng_vectorization.adapt(train_eng_texts)

Los cambios realizados con respecto al código original reflejan la adaptación para la traducción de español a inglés. La personalización en la vectorización del español, añadiendo el carácter `"¿"` a los caracteres de puntuación que se eliminan, es un ajuste específico para el idioma español, lo que mejora la calidad de la vectorización en este contexto particular.

A continuación, formatearemos nuestros conjuntos de datos.

En cada paso de entrenamiento, el modelo intentará predecir las palabras objetivo N+1 (y siguientes) utilizando la oración fuente y las palabras objetivo de 0 a N.

Por lo tanto, el conjunto de datos de entrenamiento generará una tupla `(inputs, targets)`, donde:

- `inputs` es un diccionario con las claves `encoder_inputs` y `decoder_inputs`. `encoder_inputs` es la oración fuente vectorizada y `decoder_inputs` es la oración objetivo hasta el momento, es decir, las palabras de 0 a N utilizadas para predecir la palabra N+1 (y siguientes) en la oración objetivo.
- `target` es la oración objetivo desplazada un paso: proporciona las siguientes palabras en la oración objetivo, lo que el modelo intentará predecir.

In [7]:
def format_dataset(spa, eng):
    spa = spa_vectorization(spa)
    eng = eng_vectorization(eng)
    return (
        {
            "encoder_inputs": spa,
            "decoder_inputs": eng[:, :-1],
        },
        eng[:, 1:],
    )

def make_dataset(pairs):
    spa_texts, eng_texts = zip(*pairs)
    spa_texts = list(spa_texts)
    eng_texts = list(eng_texts)
    dataset = tf_data.Dataset.from_tensor_slices((spa_texts, eng_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

En comparación con el código original, este fragmento ha sido adaptado para reflejar nuestro enfoque de traducción de español a inglés. Los `encoder_inputs` son las oraciones en español y los `decoder_inputs` son las oraciones en inglés correspondientes. Esta estructura de datos está alineada con la configuración de nuestro modelo Transformer, donde el español actúa como entrada y el inglés como objetivo de la traducción.

Echemos un vistazo rápido a las formas de las secuencias (tenemos lotes de 64 pares, y todas las secuencias tienen una longitud de 20 pasos):

In [8]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


Este fragmento de código nos permite verificar la estructura de los datos que estamos alimentando al modelo. Cada lote contiene 64 pares de oraciones, con las oraciones en español como `encoder_inputs` y las correspondientes oraciones en inglés (más los tokens `[start]` y `[end]`) como `decoder_inputs` y `targets`. Todas las secuencias están restringidas a una longitud de 20 pasos, lo que facilita el manejo uniforme de los datos en el modelo. Esta verificación es crucial para asegurarse de que los datos se estén preparando correctamente para el entrenamiento del modelo de traducción de español a inglés.

## Construcción del Modelo

Nuestro Transformer secuencia a secuencia consta de un `TransformerEncoder` y un `TransformerDecoder` encadenados. Para que el modelo sea consciente del orden de las palabras, también utilizamos una capa de `PositionalEmbedding`.

La secuencia de origen (en español) se pasará al `TransformerEncoder`, que producirá una nueva representación de la misma. Esta nueva representación se pasará luego al `TransformerDecoder`, junto con la secuencia objetivo hasta el momento (palabras objetivo de 0 a N en inglés). El `TransformerDecoder` buscará predecir las siguientes palabras en la secuencia objetivo (N+1 y siguientes).

Un detalle clave que hace esto posible es el enmascaramiento causal (véase el método `get_causal_attention_mask()` en `TransformerDecoder`). El `TransformerDecoder` ve las secuencias completas de una vez, por lo que debemos asegurarnos de que solo utilice información de los tokens objetivo de 0 a N al predecir el token N+1 (de lo contrario, podría utilizar información del futuro, lo que resultaría en un modelo que no se podría usar en el momento de la inferencia).

In [9]:
import tensorflow as tf
import keras
from keras import layers
from keras.layers import TextVectorization

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype="int32")
        else:
            padding_mask = None

        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(0, length, 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        if mask is None:
            return None
        else:
            return tf.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = None

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, None]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.convert_to_tensor([1, 1])],
            axis=0,
        )
        return tf.tile(mask, mult)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


En comparación con el código original, hemos adaptado nuestro modelo para la traducción de español a inglés. Mantenemos la arquitectura general del Transformer, pero con la particularidad de que nuestras entradas y salidas están adaptadas para trabajar con secuencias en español como entradas y sus correspondientes traducciones en inglés como objetivos. Además, el manejo del enmascaramiento causal en el `TransformerDecoder` es crucial para garantizar que la predicción de cada palabra en inglés se base solo en las palabras anteriores, y no en las futuras, lo que es esencial para la efectividad del modelo durante la inferencia.

A continuación, ensamblaremos el modelo completo de principio a fin.

In [10]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

En este paso, integramos los componentes del Transformer que hemos construido: el `TransformerEncoder`, el `TransformerDecoder` y la `PositionalEmbedding`. Los inputs del codificador son las secuencias en español, y los del decodificador son las secuencias en inglés hasta el momento. El objetivo es que el decodificador prediga la próxima palabra en la secuencia en inglés. Esta arquitectura refleja nuestra adaptación para el proyecto de traducción de español a inglés, asegurando que el modelo procese adecuadamente las secuencias de origen y genere las traducciones correspondientes.

## Entrenamiento de Nuestro Modelo

Usaremos la precisión como una forma rápida de monitorear el progreso del entrenamiento en los datos de validación. Cabe destacar que, en la traducción automática, típicamente se utilizan puntuaciones BLEU y otras métricas, en lugar de la precisión.

**Nota:** La puntuación BLEU (Bilingual Evaluation Understudy) es una métrica utilizada para evaluar la calidad de la traducción automática de textos. BLEU compara las traducciones generadas por la máquina con una o varias traducciones de referencia hechas por humanos, evaluando la similitud en términos de la precisión y fluidez de la traducción.

BLEU funciona mediante el conteo de coincidencias de frases o n-gramas (secuencias de n palabras) entre la traducción generada y las traducciones de referencia. Luego, se calcula una puntuación que considera tanto la coincidencia de palabras como la longitud de la traducción, para evitar favorecer respuestas más cortas. Las puntuaciones BLEU suelen oscilar entre 0 y 100, donde un puntaje más alto indica una mayor similitud con las traducciones de referencia y, por lo tanto, se considera una mejor traducción.

Aunque BLEU es ampliamente utilizada, tiene limitaciones, como la incapacidad para evaluar completamente la fluidez y la naturalidad del lenguaje. A pesar de esto, sigue siendo una herramienta estándar en la evaluación de modelos de traducción automática debido a su eficacia y facilidad de uso.

In [11]:
epochs = 30

# Ver el resumen del modelo
transformer.summary()

# Compilar el modelo
transformer.compile(
    optimizer="rmsprop",  # Puedes experimentar con diferentes optimizadores como 'adam'
    loss="sparse_categorical_crossentropy",  # Esta es una elección común para clasificación
    metrics=["accuracy"]  # Medir la precisión durante el entrenamiento
)

# Entrenar el modelo
transformer.fit(
    train_ds,  # Conjunto de datos de entrenamiento
    epochs=epochs,  # Número de épocas para el entrenamiento
    validation_data=val_ds  # Conjunto de datos de validación para evaluar la convergencia
)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 positional_embedding (Posi  (None, None, 256)            3845120   ['encoder_inputs[0][0]']      
 tionalEmbedding)                                                                                 
                                                                                                  
 decoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                      

Los resultados del entrenamiento del modelo Transformer para la tarea de traducción de español a inglés muestran una mejora progresiva en la precisión y una disminución en la pérdida tanto en el conjunto de entrenamiento como en el de validación a lo largo de las 30 épocas.

- **Pérdida (Loss)**: La pérdida, medida con la función de pérdida de entropía cruzada categórica dispersa, disminuye consistentemente a medida que avanza el entrenamiento. Esto indica que el modelo está aprendiendo efectivamente a predecir la siguiente palabra en las secuencias en inglés basándose en las secuencias en español y las palabras objetivo anteriores.

- **Precisión (Accuracy)**: La precisión aumenta con cada época, lo que sugiere que el modelo se está volviendo más hábil en la predicción correcta de la siguiente palabra en la secuencia. Sin embargo, es importante recordar que la precisión no es la métrica ideal para la traducción automática, ya que no captura completamente la calidad y fluidez de las traducciones. En escenarios de producción, se recomendaría usar métricas como BLEU para una evaluación más completa.

- **Validación**: El rendimiento en el conjunto de validación es un indicador crucial de cómo el modelo generalizará a nuevos datos. Aunque la pérdida en la validación tiende a ser mayor que en el entrenamiento, la diferencia no es significativamente grande, lo que es una señal positiva de que no hay un sobreajuste significativo.

En resumen, estos resultados muestran un progreso prometedor en el entrenamiento del modelo. Sin embargo, para una evaluación más completa y precisa de su capacidad de traducción, se recomienda realizar pruebas adicionales con métricas específicas de traducción automática y analizar ejemplos de traducciones generadas por el modelo.

## Decodificando Oraciones de Prueba

Finalmente, veamos cómo traducir oraciones completamente nuevas del español al inglés. Para hacerlo, simplemente alimentamos al modelo con la oración en español vectorizada, así como el token objetivo `"[start]"`. Luego, generamos repetidamente el próximo token hasta que alcancemos el token `"[end]"`.

Este proceso implica proporcionar al modelo la oración en español y pedirle que genere su traducción en inglés, comenzando con el token `"[start]"` y continuando hasta que el modelo predice el token `"[end]"`, que señala el fin de la oración en inglés. Este enfoque es típico en la inferencia secuencia a secuencia y nos permite ver cómo el modelo maneja oraciones que no ha visto durante el entrenamiento, demostrando su capacidad para generalizar y producir traducciones coherentes en situaciones de la vida real.

In [12]:
eng_vocab = eng_vectorization.get_vocabulary()
eng_index_lookup = dict(zip(range(len(eng_vocab)), eng_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = spa_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = eng_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = tf.argmax(predictions[0, i, :], axis=-1).numpy()
        sampled_token = eng_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence.lstrip("[start] ").rstrip(" [end]")

test_spa_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
    input_sentence = random.choice(test_spa_texts)
    translated = decode_sequence(input_sentence)
    print(f"Spanish: {input_sentence}")
    print(f"English: {translated}")
    print("-" * 50)

Spanish: Por la mañana el aire es fresco.
English: in the air is air is fresh air
--------------------------------------------------
Spanish: El nuevo gobierno tiene problemas financieros.
English: he new government has trouble with financial problems
--------------------------------------------------
Spanish: Tom quiso decir lo que dijo.
English: om meant what to say
--------------------------------------------------
Spanish: Él presta dinero con un alta tasa de interés.
English: he pays money with a rate in interest
--------------------------------------------------
Spanish: Ella es superficial y materialista.
English: he is shallow and say about it
--------------------------------------------------
Spanish: Somos los amigos de Tom.
English: were toms friends
--------------------------------------------------
Spanish: Esto es fabuloso.
English: his is wonderful
--------------------------------------------------
Spanish: Eso es todo lo que ella escribió.
English: hats all she wrot
---

Después de 30 épocas de entrenamiento, hemos obtenido resultados como los siguientes para la traducción de español a inglés:

> Por la mañana el aire es fresco.
> [start] In the air is air is fresh air [end]

> El nuevo gobierno tiene problemas financieros.
> [start] He new government has trouble with financial problems [end]

> Tom quiso decir lo que dijo.
> [start] Om meant what to say [end]

> Él presta dinero con un alta tasa de interés.
> [start] He pays money with a rate in interest [end]

> Somos los amigos de Tom.
> [start] Were toms friends [end]

> Esto es fabuloso.
> [start] His is wonderful [end]

> Eso es todo lo que ella escribió.
> [start] Hats all she wrot [end]

> ¿Cómo de lejos está tu casa?
> [start] Hows your house far from you [end]

> Su pasatiempo era coleccionar monedas antiguas.
> [start] Her hobby was collecting old coins [end]

> No eres joven.
> [start] Youre not young [end]

Estos ejemplos muestran cómo el modelo ha aprendido a traducir oraciones del español al inglés. Aunque en algunos casos logra traducciones coherentes, en otros hay errores o traducciones literales que podrían mejorarse.

**Análisis de los Resultados y Precisión:**

Los resultados muestran una capacidad variada del modelo para generar traducciones coherentes. Algunas oraciones son traducidas de manera bastante precisa y fluida, mientras que otras presentan errores o incoherencias. Esto puede deberse a varios factores:

1. **Limitaciones del Modelo**: Aunque el modelo ha aprendido patrones generales del lenguaje, todavía puede tener dificultades con estructuras gramaticales complejas, ambigüedades o palabras menos comunes.

2. **Tamaño del Vocabulario y Longitud de Secuencia**: El tamaño del vocabulario y la longitud máxima de secuencia pueden limitar la capacidad del modelo para manejar una variedad más amplia de frases y estructuras.

3. **Calidad del Conjunto de Datos**: Las inconsistencias o limitaciones en los datos de entrenamiento pueden afectar el rendimiento del modelo.

4. **Necesidad de Ajuste Fino**: El modelo puede beneficiarse de un ajuste fino adicional, posiblemente con un conjunto de datos más grande o diverso, o mediante ajustes en los hiperparámetros.

## Conclusiones:

- El modelo muestra un rendimiento prometedor, pero hay margen de mejora en términos de precisión y fluidez de las traducciones.
- Se podrían considerar estrategias adicionales para mejorar el rendimiento, como aumentar el tamaño del vocabulario, extender la longitud máxima de las secuencias o emplear técnicas de post-procesamiento.
- Para una evaluación más completa, sería útil utilizar métricas específicas de traducción automática, como BLEU, para obtener una medida cuantitativa de la calidad de las traducciones. 

En resumen, aunque el modelo ha aprendido a traducir de manera efectiva en muchos casos, aún hay áreas de mejora que pueden abordarse para aumentar la calidad general de las traducciones generadas.